In [32]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate , ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough , RunnableLambda , RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Markdown, display

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
loader = PyPDFLoader("../Surface_Treatment_of_Metals.pdf")
docs = loader.load()

embeddin_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000 , 
    chunk_overlap = 100
)

chunks = splitter.split_documents(docs)

In [3]:
vector_store = FAISS.from_documents(docs,embeddin_model)
retriever = vector_store.as_retriever()

In [4]:
llm = ChatGroq(model="openai/gpt-oss-120b",temperature=0)

# Query Enhancment

## 1- Query Expansion Technique

It Used Because the ...
- Original query is short ,ambiguous , or under-specified
- You want to broaden the scope to catch synonyms , related phrases , or spelling variants

In [ ]:
query_expansion_prompt = PromptTemplate.from_template(
"""you are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms , technical terms , and useful context.
Original Query : "{query}"

Expanded Query :""")

query_expansion_chain = query_expansion_prompt | llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='you are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms , technical terms , and useful context.\nOriginal Query : "{query}"\n\nExpanded Query :')
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000025AE4D260C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025AE4D43CB0>, model_name='openai/gpt-oss-120b', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [27]:
print(query_expansion_chain.invoke({"query" : "What is Metal Surface Treatment?"}))

**Expanded Query (ready for use in a search engine or academic database)**  

```
("metal surface treatment" OR "metal surface finishing" OR "metal surface modification" OR "metal surface processing")
AND
("surface coating" OR "protective coating" OR "protective film" OR "protective layer")
AND
("chemical treatment" OR "chemical passivation" OR "chemical conversion coating" OR "phosphate coating" OR "chromate conversion")
AND
("mechanical treatment" OR "mechanical polishing" OR "mechanical grinding" OR "shot peening" OR "blasting" OR "abrasive blasting")
AND
("thermal treatment" OR "heat treatment" OR "thermal oxidation" OR "annealing" OR "tempering" OR "diffusion coating")
AND
("electrochemical treatment" OR "electroplating" OR "electroless plating" OR "anodizing" OR "electro‑chemical polishing")
AND
("surface engineering" OR "surface technology" OR "surface science")
AND
("corrosion resistance" OR "wear resistance" OR "fatigue life" OR "adhesion improvement" OR "lubricity")
AND
("mic

In [28]:
answer_prompt = PromptTemplate.from_template(
"""Answer the question based on the context below.
Context : {context}

Question : {input}""")

def docs_collection (docs) : 
    return "\n\n".join(doc.page_content for doc in docs)

rag_pipeline = (
    {
        "context" : 
            RunnableLambda (
                lambda x: retriever.invoke(
                    query_expansion_chain.invoke({"query":x['input']})
                )
            )
        | docs_collection ,
        "input":RunnablePassthrough()
    }
    | answer_prompt 
    | llm 
    | StrOutputParser()
)

In [29]:
query = "What is the metal surface treatment?"
response = rag_pipeline.invoke({'input' : query})
print(display(Markdown(response)))

**Metal surface treatment** is any intentional process applied to the outermost layer of a metal in order to modify its physical, chemical or mechanical characteristics.  
In the articles gathered in this Special Issue the term encompasses a wide range of techniques, for example:

| Technique | What it does to the metal surface | Typical benefits mentioned in the editorial |
|-----------|-----------------------------------|---------------------------------------------|
| **Coating (thermal spray, atmospheric plasma spray, ZrO₂ ceramic layers, AMDRY 1371, etc.)** | Deposits a thin, adherent layer of a different material (ceramic, alloy, polymer) on the substrate. | Improves corrosion resistance, wear resistance, biocompatibility, and durability of implants or mechanical parts. |
| **Diffusion‑based nitriding (gas‑nitriding, laser‑assisted nitriding)** | Introduces nitrogen atoms into the surface, forming hard nitride phases (ε, γ′) and a diffusion zone. | Increases surface hardness, wear resistance, and fatigue strength. |
| **Laser Heat Treatment (LHT) after nitriding** | Uses a focused laser to locally re‑heat the nitrided layer, refining microstructure and relieving residual stresses. | Produces a tougher, more uniform nitrided layer with better mechanical properties. |
| **Sulphur‑diffusion blocking at the metal‑mold interface** | Controls the diffusion of sulphur during casting to limit detrimental reactions. | Leads to a thinner, more uniform casting skin and improved graphite nodularity. |
| **Thin‑film sensitisation (e.g., P3HT polymer films with FeCl₃)** | Alters optical and electrical properties of a polymer coating. | Stabilises optical performance under illumination and dark cycles. |
| **Alloying additions (e.g., 0.05 wt % Ga in Sn‑0.7Cu solder)** | Modifies the bulk and intermetallic‑compound (IMC) structure of the solder surface. | Reduces tin whisker growth, refines IMCs, and raises shear strength. |

In short, **metal surface treatment** refers to the suite of processes—coating, diffusion, laser‑assisted modification, alloying, and related methods—used to tailor a metal’s surface so that it performs better in its intended environment (e.g., resisting corrosion, wear, stress shielding, or electrical degradation).

None


## 2- Query Decomposition 

It Used Because the ...
- Complex queries often involve multiple concepts
- LLMs or retrievers may miss parts of the original question
- it enables multi-hop reasoning ((answering in steps))
- Allow parallelism (especially in multi-agent frameworks)

In [31]:
decomp_prompt = PromptTemplate.from_template(
"""You are a helpful assistant that performs query decomposition for better document retrieval.
Given the original query, break it down into multiple smaller, meaningful sub-queries that cover:
- synonyms and related terminology
- technical/scientific terms
- possible domain-specific interpretations
- relevant materials, processes, methods, or applications
- any clarifying context a search engine would find useful

Format the output as a numbered list of short decomposed sub-queries.

Original Query: "{query}"

Decomposed Sub-Queries:
1."""
)
decomp_chain = decomp_prompt | llm | StrOutputParser()
print(decomp_chain.invoke({"query" : "What is Metal Surface Treatment?"}))


1. Definition and overview of metal surface treatment (also called metal surface finishing or surface engineering)  
2. Common metal surface modification techniques: plating, anodizing, electroless plating, PVD, CVD, thermal spray, conversion coatings  
3. Mechanical surface treatments for metals: grinding, polishing, shot peening, sandblasting, laser texturing, abrasive flow machining  
4. Chemical and electrochemical surface treatments: acid pickling, passivation, phosphating, chromating, anodic oxidation, electro‑plating  
5. Thermal surface hardening processes: case hardening, carburizing, nitriding, induction hardening, flame hardening, laser hardening  
6. Surface cleaning and preparation steps prior to treatment: degreasing, solvent cleaning, ultrasonic cleaning, abrasive cleaning, etching  
7. Corrosion‑resistant surface treatments for steel, aluminum, magnesium, and copper alloys (e.g., zinc coating, zinc‑nickel alloy, Alodine, TiN coating)  
8. Wear‑resistant and tribological

In [39]:
def extract_queries(text: str):
    queries = []
    for line in text.splitlines():
        line = line.strip()
        if line and line[0].isdigit():
            cleaned = line.split(".", 1)[1].strip()
            if cleaned:
                queries.append(cleaned)
    return queries or [text]

parse_to_list = RunnableLambda(extract_queries)

In [40]:
def safe_multi_retrieve(queries):
    """Run retriever on each decomposed query, merge unique docs."""
    all_docs = []
    seen = set()
    for q in queries:
        results = retriever.invoke(q)  # each is single query now
        for doc in results:
            if doc.page_content not in seen:
                all_docs.append(doc)
                seen.add(doc.page_content)
    return all_docs

multi_retrieve = RunnableLambda(safe_multi_retrieve)


docs_to_text = RunnableLambda(
    lambda docs: "\n\n".join(doc.page_content for doc in docs)
)

answer_prompt = PromptTemplate.from_template(
    """Answer the question based on the context below.

Context:
{context}

Question:
{input}

Answer:"""
)

rag_pipeline = (
    RunnableParallel(
        {
            "context": (
                RunnableLambda(lambda x: decomp_chain.invoke({"query": x["input"]}))
                | parse_to_list
                | multi_retrieve
                | docs_to_text
            ),
            "input": RunnablePassthrough(),
        }
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [41]:
query = "What is metal surface treatment?"
response = rag_pipeline.invoke({"input": query})
print("\n=== FINAL ANSWER ===\n")
print(display(Markdown(response)))


=== FINAL ANSWER ===



Metal surface treatment refers to the variety of processes and technologies applied to the outer layer of a metal in order to modify its physical, chemical, or mechanical properties.  As highlighted in the special issue, these treatments can include:

* **Coating deposition** (thermal spray, atmospheric plasma spray, ceramic layers, ZrO₂, AMDRY 1371, etc.) to improve corrosion resistance, wear resistance, biocompatibility, and durability.  
* **Diffusion‑based treatments** such as gas nitriding followed by laser heat treatment, which create hardened compound and diffusion zones that enhance hardness and mechanical performance.  
* **Surface modification of thin films** (e.g., polymer‑based P3HT films, Sn‑Cu‑Ga solders) to stabilize optical/electrical properties or suppress tin whisker growth.  
* **Specialized treatments** like sulphur‑diffusion blocking at the metal‑mold interface to control casting skin formation.

Overall, metal surface treatment is any intentional alteration—by coating, diffusion, heat‑treatment, or other means—aimed at improving a metal’s corrosion behaviour, tribological performance, mechanical strength, biocompatibility, or other functional attributes for its intended application.

None
